In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()


# classifiers we will use
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# model selection bits
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.model_selection import learning_curve, validation_curve

# evaluation
from sklearn.metrics import f1_score

# plotting
from plotting import plot_learning_curve, plot_validation_curve

#dataset
from sklearn.datasets import load_digits


## Digit Classification

In this notebook you will practice making different ensemble classifiers working with a set of handwritten digits. Sklearn gives us a copy of the test set (so it will be smaller than decription you read below).

Our aim is to build some models and practice with the parameters. We want to adjust things like
* number of estimators
* estimators parameters

Our analysis will be very similar to the last lab, we want to visualize how the parameters are affecting the performance so we can make validation curves. 

Let's get started!


In [ ]:
data = load_digits()
data.keys()

In [ ]:
print(data.DESCR)

In [4]:
X = data.data # data is the flattened version of the images
y = data.target

## Plotting a few samples

It's always a good idea to "look" at our data. Our data is handwritten digits and in fact that means they are stored as 8x8 matrices.  This is stored in `data.images`.  So let's plot a few of those and see what they look like.

The astute reader will notice that we have used this code before, in fact we looked at this dataset in the intro notebook.

In [ ]:
print(data.images[0])


In [ ]:
plt.imshow(data.images[0], cmap='binary', interpolation='nearest');


In [ ]:
fig, axes = plt.subplots(2, 10, figsize=(12, 2))

for i, ax in enumerate(axes.flat):
    ax.imshow(data.images[i], cmap='binary', interpolation='nearest')
    ax.text(0.05, 0.05, str(data.target[i]),
            transform=ax.transAxes, color='green')
    ax.set_xticks([])
    ax.set_yticks([])

Okay! Some blurry images.  However, we won't be doing our classification on 8x8 pictures, because all the algorithms we currently know need row vector samples, we will need to `flatten` our images into long vectors that are (64,) in shape (if this sounds insane to you, it is -- but it works surprisingly well, however you would like to learn convolutional neural networks (CNN's) in your future!).  But sklearn is so kind that they have already done this for us, so we just take our data from `data.data` (instead of `data.images`)

### Explore the Data

Let's investigate the data a bit more. Answer the following questions

* Are there any missing data points or features? (probably not, but you should always check!)
* How many classes are there?
* What does the class distribution look like?

### On your Models, get set, GO!

Go ahead and make a your models.  We want AdaboostClassifier and a RandomForestClassifier.
Go ahead and read the documentation for both of these models and try out the different parameters.

I suggest you try and plot 3 validation curves and 1 learning curve for each model.  How well do they perform? Which one is better? What surprised you?

What scoring metric is appropriate for this task? We've been using the f1-score for binary classification, but this is a little different.
[Take a read on this thread if you run into any problems](https://forum.codingnomads.co/t/target-is-multiclass-but-average-binary-please-choose-another-average-setting-one-of-none-micro-macro-weighted/811)

Also, how many trees will you be creating per graph?

# Wait ! One more Thing!
Before I leave you to it, I need to show you how to set parameters for estimators. Since adaboost can use any estimator, we may want to try different parameters on it's estimator.

The easy way to do this is as follows:

```
my_decision_tree = DecisionTreeClassifier(max_depth=10, min_samples_leaf=20)
ada = AdaBoostClassifier(my_decision_tree, n_estimators=100)
```

However, what do we do if we want to run a grid-search or validation curve and change the parameters of the estimator?  Then we have a special syntax from sklearn.  It works like this


```
ada = AdaBoostClassifier(estimator=DecisionTreeClassifier())
param_grid = {'estimator__max_depth':[2,10,20],
              'estimator__min_samples_leaf':[2,10,20]}
grid = GridSearchCV(ada, param_grid, cv=3)
```

Note that the estimator keyword for Adaboost is `estimator`. So we access the parameters of the `estimator` using the double under `__` and then put the our known estimators.  This kind of "digging" to access parameters is a default API in sklearn. In more complicated situations you can go multiple layers, the doucmentation provides nice examples here:

https://scikit-learn.org/stable/modules/grid_search.html#composite-estimators-and-parameter-spaces


Ok, now you know enough to be dangerous, so let's make some models!
I suggest you try at least 3 validation curves for adaboost and randonforests.  Since they can both share the same estimator you can try and compare how the same settings affect each learning algorithm differently.  